In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))
import os

In [2]:
import hail as hl

localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'
hl.init(tmp_dir=localfs_path, local_tmpdir=localfs_path, 
        spark_conf={'spark.driver.memory': '15G', 'spark.executor.memory': '30G'})

2022-10-01 14:18:45 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-10-01 14:18:46 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.3, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-10-01 14:18:46 WARN  SparkConf:69 - Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


Running on Apache Spark version 3.1.2
SparkUI available at http://p1910.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/analysis/hail-20221001-1418-0.2.97-937922d7f46c.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats
import seaborn as sns

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

### The matrix table below has already been filtered and annotated, see preprocessing/joint-with-gts/genotype-and-filter.ipynb

In [12]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full-healthy.mt')

In [13]:
mt.count()

(2639210, 237)

In [14]:
proper_controls = ['WGS_37b', 'WGS_37c', 'WGS_163d', 'WGS_7120', 'WGS_7142', 'WGS_7143', 'WGS_7152',
 'WGS_7153', 'WGS_85b', 'WGS_147c', 'WGS_180b', 'WGS_185c', 'WGS_6819', 'S_7213', 'S_7227', 'S_7241', 'S_7246', 'S_7254', 'S_7274',
                   'S_7307', '494', '462', '468', '492', '490'] 

In [15]:
len(proper_controls)

25

### Only keep intragenic variants

In [16]:
mt = mt.filter_rows(mt.within_gene == hl.empty_array(hl.tstr), keep = False)
mt.write(localfs_path+'intragenic.mt')

2022-09-29 18:22:24 Hail: INFO: wrote matrix table with 1683507 rows and 237 columns in 14999 partitions to /localfs/27722482/intragenic.mt


In [17]:
mt = hl.read_matrix_table(localfs_path+'intragenic.mt')

In [18]:
mt.count()

(1683507, 237)

### Only keep proprerly healthy controls

In [19]:
mt.aggregate_cols(hl.agg.counter(mt.group))

frozendict({'1kg': 98, 'control': 39, 'sport': 100})

In [20]:
mt = mt.filter_cols(
    (mt.group == '1kg') | (mt.group == 'sport') | (hl.literal(proper_controls)).contains(mt.s)
)

In [21]:
mt.write(localfs_path+'filtered.mt')
mt = hl.read_matrix_table(localfs_path+'filtered.mt')

2022-09-29 18:40:07 Hail: INFO: wrote matrix table with 1683507 rows and 220 columns in 14999 partitions to /localfs/27722482/filtered.mt


In [22]:
mt.count()

(1683507, 220)

In [23]:
mt.naive_coalesce(200).write(localfs_path+'repartitioned.mt')

2022-09-29 18:54:06 Hail: INFO: wrote matrix table with 1683507 rows and 220 columns in 200 partitions to /localfs/27722482/repartitioned.mt


In [24]:
mt = hl.read_matrix_table(localfs_path+'repartitioned.mt')

## calculate per-gene-per-group-burden in two ways:
 - sum of CADD scores for CADD variants with scores > 20
 - LoF - HC lof from vep

In [33]:
mt = mt.explode_rows(mt.within_gene)
mt.write(localfs_path+'exploded.mt')

2022-09-29 19:10:43 Hail: INFO: wrote matrix table with 1915897 rows and 220 columns in 200 partitions to /localfs/27722482/exploded.mt


In [4]:
mt = hl.read_matrix_table(localfs_path+'exploded.mt')

In [6]:
burden_cadd = mt.filter_rows(mt.cadd.cadd_score > 20) #option 1
burden_cadd = burden_cadd.annotate_entries(cadd_entry = burden_cadd.GT.n_alt_alleles()*burden_cadd.cadd.cadd_score)
burden_cadd.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_cadd.mt')

2022-09-29 19:18:49 Hail: INFO: wrote matrix table with 16452 rows and 220 columns in 200 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_cadd.mt


In [23]:
burden_lof = mt.filter_rows(mt['gnomad_v_3_1']['vep']['transcript_consequences']['lof'].contains('HC')) #option 2
burden_lof.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_lof.mt')

2022-09-29 19:31:41 Hail: INFO: wrote matrix table with 1426 rows and 220 columns in 200 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_lof.mt


In [5]:
b_cadd = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_cadd.mt')
b_lof = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_lof.mt')

In [6]:
per_gene_cadd = b_cadd.group_rows_by(b_cadd.within_gene).aggregate(
    burden_cadd_phred = hl.agg.sum(b_cadd.cadd_entry)
)

per_gene_lof = b_lof.group_rows_by(b_lof.within_gene).aggregate(
    lof = hl.agg.sum(b_lof.GT.n_alt_alleles()
                    )
)

per_gene_cadd = per_gene_cadd.filter_cols(per_gene_cadd.s != '494')
per_gene_lof = per_gene_lof.filter_cols(per_gene_lof.s != '494')

#opcjonalnie jeżeli będzie taka potrzeba

In [7]:
per_gene_cadd.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_cadd.mt')
per_gene_lof.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_lof.mt')

2022-10-01 14:20:45 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2022-10-01 14:20:51 Hail: INFO: wrote matrix table with 7432 rows and 219 columns in 200 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_cadd.mt
2022-10-01 14:20:52 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2022-10-01 14:20:54 Hail: INFO: wrote matrix table with 1018 rows and 219 columns in 200 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_lof.mt


### Calculate burden statistics

In [8]:
per_gene_cadd = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_cadd.mt')
per_gene_lof = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_lof.mt')

In [9]:
per_gene_cadd.count()

(7432, 219)

In [10]:
per_gene_lof.count()

(1018, 219)

In [11]:
# getting the 
gtex = pd.read_table(
    '/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct',
    index_col=1
)

gtex = gtex.drop('Name', axis=1)
gtex = gtex.stack()
gtex = gtex[gtex > 100]
gtex.index.names = ['gene_name', 'tissue']
gtex = pd.DataFrame(gtex, columns=['value'])
gtex = gtex.reset_index()
gtex = gtex.drop('value', axis=1)
gtex = hl.Table.from_pandas(gtex)
gtex = gtex.checkpoint(localfs_path+'gtex-5.ht')
gtex.show()

2022-10-01 14:22:11 Hail: INFO: wrote table with 47704 rows in 16 partitions to /localfs/27822732/gtex-5.ht


,
gene_name,tissue
str,str
"""RP11-34P13.15""","""Whole Blood"""
"""RP11-34P13.16""","""Whole Blood"""
"""MTND2P28""","""Adipose - Subcutaneous"""
"""MTND2P28""","""Adipose - Visceral (Omentum)"""
"""MTND2P28""","""Adrenal Gland"""
"""MTND2P28""","""Artery - Aorta"""
"""MTND2P28""","""Artery - Coronary"""
"""MTND2P28""","""Artery - Tibial"""


In [12]:
tissue = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/normal_tissue.tsv')
high = tissue.filter(tissue.Level == 'High')
high_agg = high.group_by('Tissue').aggregate(n=hl.agg.count())
high = high.annotate(
    **high_agg[high.Tissue]
)
high = high.filter(high.n > 50)
high = high.select(gene_name=high['Gene name'], tissue=high['Tissue'])
high = high.checkpoint(localfs_path+'tissues-5.ht')
high.show()

tissues = gtex.union(high)
tissues = tissues.checkpoint(localfs_path+'gene-lists-5.ht')
tissues.show()

2022-10-01 14:22:13 Hail: INFO: Reading table without type imputation
  Loading field 'Gene' as type str (not specified)
  Loading field 'Gene name' as type str (not specified)
  Loading field 'Tissue' as type str (not specified)
  Loading field 'Cell type' as type str (not specified)
  Loading field 'Level' as type str (not specified)
  Loading field 'Reliability' as type str (not specified)
2022-10-01 14:22:14 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-10-01 14:22:16 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-10-01 14:22:19 Hail: INFO: wrote table with 139162 rows in 1 partition to /localfs/27822732/tissues-5.ht


,
gene_name,tissue
str,str
"""SEMA3F""","""adipose tissue"""
"""TMEM176A""","""adipose tissue"""
"""M6PR""","""adipose tissue"""
"""RBM5""","""adipose tissue"""
"""HSPB6""","""adipose tissue"""
"""RPAP3""","""adipose tissue"""
"""PRKAR2B""","""adipose tissue"""
"""AP2B1""","""adipose tissue"""


2022-10-01 14:22:20 Hail: INFO: wrote table with 186866 rows in 1 partition to /localfs/27822732/gene-lists-5.ht


,
gene_name,tissue
str,str
"""RP11-34P13.15""","""Whole Blood"""
"""RP11-34P13.16""","""Whole Blood"""
"""MTND2P28""","""Adipose - Subcutaneous"""
"""MTND2P28""","""Adipose - Visceral (Omentum)"""
"""MTND2P28""","""Adrenal Gland"""
"""MTND2P28""","""Artery - Aorta"""
"""MTND2P28""","""Artery - Coronary"""
"""MTND2P28""","""Artery - Tibial"""


In [13]:
# annotate per_gene lof and cadd table with tissues

tissues = tissues.group_by('gene_name').aggregate(tissue=hl.agg.collect_as_set(tissues.tissue))
tissues = tissues.key_by('gene_name')
tissues = tissues.checkpoint(localfs_path+'tissues-6.ht')

pgc = per_gene_cadd.annotate_rows(**tissues[per_gene_cadd.within_gene])
pgc = pgc.explode_rows(pgc.tissue)
pgc = pgc.checkpoint(localfs_path+'pgc-5.ht')

pgl = per_gene_lof.annotate_rows(**tissues[per_gene_lof.within_gene])
pgl = pgl.explode_rows(pgl.tissue)
pgl = pgl.checkpoint(localfs_path+'pgl-5.ht')

2022-10-01 14:22:23 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-10-01 14:22:24 Hail: INFO: wrote table with 12529 rows in 1 partition to /localfs/27822732/tissues-6.ht
2022-10-01 14:22:26 Hail: INFO: wrote matrix table with 41512 rows and 219 columns in 200 partitions to /localfs/27822732/pgc-5.ht
2022-10-01 14:22:28 Hail: INFO: wrote matrix table with 5531 rows and 219 columns in 200 partitions to /localfs/27822732/pgl-5.ht


In [14]:
# compute burden per tissue:

agc = pgc.group_rows_by(pgc.tissue)
agc = agc.aggregate_entries(result_score = hl.agg.sum(agc['burden_cadd_phred']))
agc = agc.result().checkpoint(localfs_path+'agc-5.ht')

agl = pgl.group_rows_by(pgl.tissue)
agl = agl.aggregate_entries(result_score = hl.agg.sum(agl['lof']))
agl = agl.result().checkpoint(localfs_path+'agl-5.ht')

2022-10-01 14:22:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-10-01 14:22:31 Hail: INFO: wrote matrix table with 104 rows and 219 columns in 96 partitions to /localfs/27822732/agc-5.ht
2022-10-01 14:22:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-10-01 14:22:34 Hail: INFO: wrote matrix table with 104 rows and 219 columns in 89 partitions to /localfs/27822732/agl-5.ht


In [16]:
# pandas export results for both types and annotate them with group and phenotype:

ag_pds = []

for ag in [agl, agc]:
    ag_pd = ag.select_cols('group', type=ag.sport_phenotypes.type)
    ag_pd = ag_pd.entries()
    ag_pd = ag_pd.to_pandas()
    ag_pd = ag_pd.drop('s', axis=1)
    
    # oraz połącz kolumnę group i pheontype type
    ag_pd_group = ag_pd.drop('type', axis=1)
    ag_pd_type = ag_pd.drop('group', axis=1)
    ag_pd_type = ag_pd_type[~ag_pd_type.type.isna()]
    ag_pd_type = ag_pd_type.rename({'type': 'group'}, axis=1)
    ag = pd.concat([ag_pd_group, ag_pd_type])

    ag_pds.append(ag)

# calculate statistics for each of the tissues:

def ttest(df, x, y):
    pval = stats.ttest_ind(
        df[df.group == x]['result_score'],
        df[df.group == y]['result_score']
    ).pvalue
    return pval

outs = []

for ag in ag_pds:
    ag_grouped = ag.groupby('tissue')

    out = pd.DataFrame({
        'p_value_gts_sport': ag_grouped.apply(lambda df: ttest(df, 'control', 'sport')),
        'p_value_gts_1kg': ag_grouped.apply(lambda df: ttest(df, 'control', '1kg')),
        'p_value_1kg_sport': ag_grouped.apply(lambda df: ttest(df, '1kg', 'sport')),
        'p_value_end_vs_speed': ag_grouped.apply(lambda df: ttest(df, 'endurance', 'speed')),
        'gts_mean': ag_grouped.apply(lambda df: df[df.group == 'control'].result_score.mean()),
        'gts_std': ag_grouped.apply(lambda df: df[df.group == 'control'].result_score.std()),
        'kg_mean': ag_grouped.apply(lambda df: df[df.group == '1kg'].result_score.mean()),
        'kg_std': ag_grouped.apply(lambda df: df[df.group == '1kg'].result_score.std()),
        'sport_mean': ag_grouped.apply(lambda df: df[df.group == 'sport'].result_score.mean()),
        'sport_std': ag_grouped.apply(lambda df: df[df.group == 'sport'].result_score.std()),
        'end_mean': ag_grouped.apply(lambda df: df[df.group == 'endurance'].result_score.mean()),
        'end_std': ag_grouped.apply(lambda df: df[df.group == 'endurance'].result_score.std()),
        'speed_mean': ag_grouped.apply(lambda df: df[df.group == 'speed'].result_score.mean()),
        'speed_std': ag_grouped.apply(lambda df: df[df.group == 'speed'].result_score.std()),
    })
    
    outs.append(out)

In [17]:
outs[0]

,p_value_gts_sport,p_value_gts_1kg,p_value_1kg_sport,p_value_end_vs_speed,gts_mean,gts_std,kg_mean,kg_std,sport_mean,sport_std,end_mean,end_std,speed_mean,speed_std
tissue,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.258393,0.990119,0.048429,0.204225,9.809524,2.821685,9.816327,2.150746,10.41,2.055272,10.142857,2.051422,10.666667,2.046135
Adipose - Visceral (Omentum),0.679295,0.391909,0.023320,0.232244,8.666667,2.670830,8.224490,2.013002,8.88,2.021501,8.632653,2.176521,9.117647,1.850914
Adrenal Gland,0.779640,0.287964,0.022519,0.419997,6.000000,1.897367,5.642857,1.262055,6.11,1.575667,5.979592,1.574402,6.235294,1.582255
Artery - Aorta,0.310097,0.806097,0.033814,0.480654,13.809524,2.676174,13.663265,2.428117,14.36,2.153316,14.204082,2.071429,14.509804,2.239398
Artery - Coronary,0.257961,0.879833,0.030255,0.441440,13.619048,2.692140,13.530612,2.369062,14.23,2.136054,14.061224,2.024930,14.392157,2.245693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
testis,0.842995,0.032098,0.000580,0.103177,53.523810,5.519230,56.173469,4.984565,53.21,6.781280,52.081633,6.006374,54.294118,7.346548
thyroid gland,0.899724,0.185238,0.012784,0.427703,21.523810,3.414953,22.500000,2.964567,21.43,3.025998,21.183673,2.905981,21.666667,3.147486
tonsil,0.510513,0.222001,0.000857,0.283688,28.904762,5.439713,30.122449,3.797083,28.21,4.139719,27.755102,4.140907,28.647059,4.131942


In [18]:
outs[1]

,p_value_gts_sport,p_value_gts_1kg,p_value_1kg_sport,p_value_end_vs_speed,gts_mean,gts_std,kg_mean,kg_std,sport_mean,sport_std,end_mean,end_std,speed_mean,speed_std
tissue,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.634220,0.471485,0.753005,0.719674,1495.233333,240.128421,1469.726531,118.811443,1475.826,151.265147,1470.246939,154.342715,1481.186275,149.586259
Adipose - Visceral (Omentum),0.260608,0.059104,0.267815,0.284980,1236.176190,213.782742,1173.471429,114.689830,1193.840,141.535958,1178.328571,146.511302,1208.743137,136.363226
Adrenal Gland,0.442353,0.478798,0.865015,0.440011,834.114286,178.486093,814.574490,95.957179,812.174,102.305826,804.065306,117.175053,819.964706,86.119080
Artery - Aorta,0.792450,0.352147,0.284189,0.112270,1831.866667,240.283731,1794.948980,143.787941,1819.782,179.278583,1790.702041,179.704965,1847.721569,176.101453
Artery - Coronary,0.479179,0.126009,0.141286,0.538695,1689.861905,240.212673,1629.271429,142.654075,1660.532,154.780576,1650.759184,144.384023,1669.921569,165.040940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
testis,0.747894,0.227419,0.238061,0.718674,9834.185714,457.678021,9939.248980,336.449431,9870.377,470.056137,9852.975510,329.550878,9887.096078,576.753833
thyroid gland,0.182968,0.034331,0.465509,0.220897,4770.009524,210.972230,4880.045918,214.269490,4854.553,272.240287,4820.424490,222.193422,4887.343137,311.631153
tonsil,0.793097,0.146728,0.062814,0.265306,6383.957143,282.775873,6478.769388,267.164358,6402.836,302.386048,6368.318367,242.655962,6436.000000,349.613747


In [19]:
outs[0].to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/tissues-burden/tissues-burden-cadd.csv')
outs[1].to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/tissues-burden/tissues-burden-lof.csv')

In [25]:
counts = tissues.explode(tissues.tissue)

In [29]:
counts.aggregate(hl.agg.counter(counts.tissue))

frozendict({'Adipose - Subcutaneous': 1049, 'Adipose - Visceral (Omentum)': 977, 'Adrenal Gland': 791, 'Artery - Aorta': 1286, 'Artery - Coronary': 1159, 'Artery - Tibial': 1271, 'Bladder': 978, 'Brain - Amygdala': 473, 'Brain - Anterior cingulate cortex (BA24)': 590, 'Brain - Caudate (basal ganglia)': 516, 'Brain - Cerebellar Hemisphere': 1219, 'Brain - Cerebellum': 1146, 'Brain - Cortex': 689, 'Brain - Frontal Cortex (BA9)': 816, 'Brain - Hippocampus': 464, 'Brain - Hypothalamus': 526, 'Brain - Nucleus accumbens (basal ganglia)': 579, 'Brain - Putamen (basal ganglia)': 454, 'Brain - Spinal cord (cervical c-1)': 715, 'Brain - Substantia nigra': 479, 'Breast - Mammary Tissue': 940, 'Cells - Cultured fibroblasts': 1276, 'Cells - EBV-transformed lymphocytes': 1221, 'Cervix - Ectocervix': 1214, 'Cervix - Endocervix': 1253, 'Colon - Sigmoid': 966, 'Colon - Transverse': 746, 'Esophagus - Gastroesophageal Junction': 943, 'Esophagus - Mucosa': 950, 'Esophagus - Muscularis': 935, 'Fallopian Tu